In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('groupBy_and_join').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/11 18:58:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df_green_revenue = spark.sql("""

select 
    PULocationID as zone,
    date_trunc('hour', lpep_pickup_datetime) as hour,
    SUM(total_amount) as amount,
    count(1) as records
FROM
    green
where 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
Group By 
    1, 2
order by
    2, 1
""")

In [6]:
df_green_revenue.show()

+----+-------------------+------------------+-------+
|zone|               hour|            amount|records|
+----+-------------------+------------------+-------+
|   7|2020-01-01 00:00:00| 769.7299999999996|     45|
|  17|2020-01-01 00:00:00|195.03000000000003|      9|
|  18|2020-01-01 00:00:00|               7.8|      1|
|  22|2020-01-01 00:00:00|              15.8|      1|
|  24|2020-01-01 00:00:00|              87.6|      3|
|  25|2020-01-01 00:00:00| 531.0000000000002|     26|
|  29|2020-01-01 00:00:00|              61.3|      1|
|  32|2020-01-01 00:00:00| 68.94999999999999|      2|
|  33|2020-01-01 00:00:00|317.27000000000004|     11|
|  35|2020-01-01 00:00:00|            129.96|      5|
|  36|2020-01-01 00:00:00|295.34000000000003|     11|
|  37|2020-01-01 00:00:00|            175.67|      6|
|  38|2020-01-01 00:00:00| 98.78999999999999|      2|
|  40|2020-01-01 00:00:00|168.97999999999996|      8|
|  41|2020-01-01 00:00:00|1363.9599999999987|     84|
|  42|2020-01-01 00:00:00| 7

In [8]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

In [9]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

df_yellow_revenue = spark.sql("""

select 
    PULocationID as zone,
    date_trunc('hour', tpep_pickup_datetime) as hour,
    SUM(total_amount) as amount,
    count(1) as records
FROM
    yellow
where 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
Group By 
    1, 2
order by
    2, 1
""")


/home/jagadish/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

In [11]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour', 'zone'], how='outer')

In [12]:
df_join.show()

+-------------------+----+------+-------+------------------+-------+
|               hour|zone|amount|records|            amount|records|
+-------------------+----+------+-------+------------------+-------+
|2020-01-01 05:00:00|   1|  null|   null|              40.3|      1|
|2020-01-01 07:00:00|   1|  null|   null|              0.31|      1|
|2020-01-01 11:00:00|   1|  null|   null|              90.3|      1|
|2020-01-01 12:00:00|   1|  null|   null|            266.66|      3|
|2020-01-01 15:00:00|   1|  null|   null|            315.21|      4|
|2020-01-01 18:00:00|   1|  null|   null|            400.41|      3|
|2020-01-02 01:00:00|   1|  null|   null|             84.36|      1|
|2020-01-02 06:00:00|   1|  null|   null|            249.95|      3|
|2020-01-02 09:00:00|   1|  null|   null|              86.8|      1|
|2020-01-02 15:00:00|   1|  null|   null|246.60000000000002|      2|
|2020-01-02 18:00:00|   1|  null|   null|             389.5|      4|
|2020-01-03 00:00:00|   1|  null| 

In [13]:
df_green_revenue_tmp = df_green_revenue.withColumnRenamed('amount', 'green_amount').withColumnRenamed('records', 'green_records')

df_yellow_revenue_tmp = df_yellow_revenue.withColumnRenamed('amount', 'yellow_amount').withColumnRenamed('records', 'yellow_records')


In [14]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [15]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_records: bigint, yellow_amount: double, yellow_records: bigint]

In [16]:
df_join.write.parquet('data/report/revenue/total')

In [17]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [18]:
df_join.show()

+-------------------+----+------------+-------------+------------------+--------------+
|               hour|zone|green_amount|green_records|     yellow_amount|yellow_records|
+-------------------+----+------------+-------------+------------------+--------------+
|2020-01-01 06:00:00|   1|        null|         null|            445.28|             4|
|2020-01-01 10:00:00|   1|        null|         null|             100.6|             2|
|2020-01-01 14:00:00|   1|        null|         null|             239.5|             3|
|2020-01-01 16:00:00|   1|        null|         null|            574.02|             6|
|2020-01-01 19:00:00|   1|        null|         null|             96.35|             1|
|2020-01-02 07:00:00|   1|        null|         null|             95.82|             1|
|2020-01-02 08:00:00|   1|        null|         null|               0.0|             2|
|2020-01-02 14:00:00|   1|        null|         null|            628.52|             6|
|2020-01-03 14:00:00|   1|      

In [19]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [21]:
df_result.show()

+-------------------+----+------------+-------------+------------------+--------------+----------+-------+--------------+------------+
|               hour|zone|green_amount|green_records|     yellow_amount|yellow_records|LocationID|Borough|          Zone|service_zone|
+-------------------+----+------------+-------------+------------------+--------------+----------+-------+--------------+------------+
|2020-01-01 06:00:00|   1|        null|         null|            445.28|             4|         1|    EWR|Newark Airport|         EWR|
|2020-01-01 10:00:00|   1|        null|         null|             100.6|             2|         1|    EWR|Newark Airport|         EWR|
|2020-01-01 14:00:00|   1|        null|         null|             239.5|             3|         1|    EWR|Newark Airport|         EWR|
|2020-01-01 16:00:00|   1|        null|         null|            574.02|             6|         1|    EWR|Newark Airport|         EWR|
|2020-01-01 19:00:00|   1|        null|         null|  

In [22]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')